In [ ]:
import ROOT

In [ ]:
def loadHists(histFile, query=None):
    f = ROOT.TFile.Open(histFile)
    histList = {}
    keyList = f.GetListOfKeys()
    for key in keyList:
        # if query is not None and key.GetName() not in query:
        #    continue
        hist = f.Get(key.GetName())
        hist.SetDirectory(ROOT.gROOT)
        hist.SetName(key.GetName())
        histList[key.GetName()] = hist
    if len(histList) == 0: raise Exception('ERROR: histList is empty!')
    f.Close()
    return histList

In [ ]:
hist_file = '/eos/user/f/falicant/RUN1/brick21/hist_couples_aligned_mos.root'
hist_cp = loadHists(hist_file)

In [ ]:
rad_len = [round(p / 3.504,1) for p in range(58-12)]

In [ ]:
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetFrameLineWidth(2)
ROOT.gStyle.SetOptTitle(1)
ROOT.gStyle.SetTitleFontSize(0.1)

c = ROOT.TCanvas("c","c",1500,1000)
c.Divide(5,3)
i=0

plist= [12,15,19,21,25,27,29, 33,35,42,46,48,51,53,57]
for p in range(1, 58):
    if not p in plist: continue
    i+=1
    c.cd(i)
    hist_cp[f'h_{p}'].SetTitle(f"{rad_len[p-12]} X0")
    hist_cp[f'h_{p}'].SetNdivisions(4,"XYZ")
    hist_cp[f'h_{p}'].SetLabelSize(0.04,"XYZ")
    hist_cp[f'h_{p}'].SetTitleSize(13,"XY")
    hist_cp[f'h_{p}'].SetTitleFont(43,"XY")
    hist_cp[f'h_{p}'].SetTitleOffset(1.1,"XY")
    hist_cp[f'h_{p}'].GetZaxis().SetRangeUser(0,20)
    hist_cp[f'h_{p}'].Draw("COLZ")
c.Draw()

In [ ]:
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetFrameLineWidth(2)
c = ROOT.TCanvas("c","c",1500,4200)
c.Divide(5,12)
i=0

for p in list(range(1, 11))+list(range(12, 58)):
    i+=1
    c.cd(i)
    hist_cp[f'h_{p}'].SetTitle(f"Plate {p}")
    hist_cp[f'h_{p}'].GetYaxis().SetTitleOffset(1.25)
    hist_cp[f'h_{p}'].GetZaxis().SetRangeUser(0,20)
    hist_cp[f'h_{p}'].Draw("COLZ")
c.Draw()

In [ ]:
from ctypes import c_int
plate=38
cx = ROOT.TCanvas("cx","cx",500,500)
_sliceMax=c_int(0)
_sliceMay=c_int(0)
_sliceMaz=c_int(0)
hist_cp[f'h_{plate}'].GetXaxis().SetRangeUser(11500+plate*10,13000+plate*10)
hist_cp[f'h_{plate}'].GetYaxis().SetRangeUser(104000+plate*10,106000+plate*10)
hist_cp[f'h_{plate}'].GetMaximumBin(_sliceMax, _sliceMay, _sliceMaz)
sliceMax=_sliceMax.value
sliceMay=_sliceMay.value
sliceMaz=_sliceMaz.value

hx = hist_cp[f'h_{plate}'].ProjectionX("",sliceMay-5,sliceMay+5,"")
hx.Draw()
maxBinX = hx.GetMaximumBin()
maxValueX = hx.GetBinCenter(maxBinX)
hx.GetXaxis().SetRangeUser(maxValueX-500,maxValueX+500)
hx.Draw()
fx = ROOT.TF1('f1','gaus')
fitX = hx.Fit(fx,'','',maxValueX-250,maxValueX+250)
integralX = fx.Integral(maxValueX-250,maxValueX+250)
print(int(hx.GetMean()), int(hx.GetStdDev()), int(hx.GetEntries()))
cx.Draw()

In [ ]:
cy = ROOT.TCanvas("cy","cy",500,500)
hy = hist_cp[f'h_{plate}'].ProjectionY("",sliceMax-5,sliceMax+5,"")
hy.Draw()
maxBinY = hy.GetMaximumBin()
maxValueY = hy.GetBinCenter(maxBinY)
hy.GetXaxis().SetRangeUser(maxValueY-300,maxValueY+300)
fitY = hy.Fit('gaus','','',maxValueY-250,maxValueY+250)
cy.Draw()

In [ ]:
z = {57: 34655.96, 56: 33329.90, 55: 32006.92, 54: 30685.82, 53: 29372.04, 52: 28061.45, 51: 26715.19, 50: 25373.10, 49: 24062.98, 48: 22730.45, 47: 21352.18, 46: 19961.56,
    45: 18634.26, 44: 17268.02, 43: 15889.72, 42: 14523.08, 41: 13210.54, 40: 11906.15, 39: 10572.31, 38:  9258.64, 37:  7942.98, 36:  6613.56, 35:  5267.96, 34:  3965.65,
    33:  2663.28, 32:  1344.32, 31:0.00, 30: -1317.17, 29: -2660.18, 28: -3936.19, 27: -5261.42, 26: -6600.40, 25: -7903.31, 24: -9317.59, 23:-10680.85, 22:-11971.73,
    21:-13345.02, 20:-14675.77, 19:-16021.16, 18:-17358.80, 17:-18693.34, 16:-20052.59, 15:-21400.94, 14:-22716.23, 13:-24059.38, 12:-25422.78, 11:-26722.78, 10:-28046.87,
    9:-29377.34,  8:-30695.57,  7:-32045.26,  6:-33379.57,  5:-34737.75,  4:-36102.56,  3:-37462.51,  2:-38769.36,  1:-40109.79}
z.update((x, y-34655.96) for x, y in z.items())

qx = 13140.1
mx = 0.0156804
qy = 106366
my = 0.0339451
eqx = 70.0674
emx = 0.00220214 
eqy = 70.7743
emy = 0.00223281

resx = 0.6914
resy = 0.5879
restx = 0.001342
resty = 0.001175

dz=z[18]-z[1]
dz2=dz**2
eqx2=eqx**2
eqy2=eqy**2
emx2=emx**2
emy2=emy**2

resx2=resx**2
resy2=resy**2
restx2=restx**2
resty2=resty**2

errqx = eqx2+dz2*emx2
errqy = eqy2+dz2*emy2
errresx = resx2+dz2*restx2
errresy = resy2+dz2*resty2

errx = ROOT.TMath.Sqrt(errqx+errresx)
erry = ROOT.TMath.Sqrt(errqy+errresy)
errtx = ROOT.TMath.Sqrt(emx2+restx2)
errty = ROOT.TMath.Sqrt(emy2+resty2)

In [ ]:
x1 = qx+mx*z[5]
x20 = qx+mx*z[20]
y1 = qy+my*z[5]
y20 = qy+my*z[20]
print("x",round(x1,0), round(x20,0), round(errx,0))
print("y",round(y1,0), round(y20,0), round(erry,0))
print("tx",round(mx,3), round(errtx,3))
print("ty",round(my,3), round(errty,3))

In [ ]:
print("x",round(x1+(x20-x1)/2,0), round(((x20-x1)+errx*5)/2,0))
print("x",round(y1+(y20-y1)/2,0), round(((y20-y1)+erry*5)/2,0))

In [ ]:
p=17
vx=12197.6
vy=104323.4
vz = -25666.6-34655.96
print(vz)
predvx = qx+mx*vz
predvy = qy+my*vz
print(predvx,predvy)
print(vx-predvx, vy-predvy)

In [ ]:
z={57:0.00 ,56:-1340.35,55:-2714.28,54:-4036.24,53:-5337.83,52:-6684.41,51:-8001.78, 50:-9322.46 ,49:-10655.76, 48:-12021.93, 47:-13412.77, 46: -14795.63, 45:-16162.66, 44:-17534.37, 43:-18891.43, 42:-20246.43, 41:-21566.47 ,40:-22913.61 ,39:-24230.69 ,38:-25575.51 ,
   37:-26879.11 ,36:-28229.82 ,35:-29581.09 ,34:-30901.87 ,33:-32245.53 ,32:-33597.76 ,31:-34976.27 ,30:-36277.15 ,29:-37604.21 ,28:-38917.36 ,27:-40238.69, 26:-41584.73 ,25:-42908.92 ,24:-44276.78 ,23:-45643.12 ,22:-46972.20 ,21:-48355.71 ,20:-49722.71 ,19:-51066.87 ,
   18:-52427.69 ,17:-53749.28 ,16:-55136.12 ,15:-56493.45 ,14:-57828.26 ,13:-59199.22 ,12:-60574.84 ,11:-61938.31 ,10:-63250.14 , 9:-64578.07 , 8:-65892.94 , 7:-67241.09 , 6:-68555.77 , 5:-69898.01 , 4:-71225.37 , 3:-72564.90 , 2:-73891.68 , 1:-75247.02}
for key in range(1,12):
    z.pop(key, None)
sorted_z = sorted(z.values())
start_z = [sort_z - sorted_z[0] for sort_z in sorted_z]
rad_len = [round(z / 3504,1) for z in start_z]
print(start_z[26])
print(rad_len)

In [ ]:
mcfile='/eos/experiment/sndlhc/MonteCarlo/Neutrinos/Genie/nu_sim_activeemu_withcrisfiles_25_July_2022/sndLHC.Genie-TGeant4.root'
mcfile=ROOT.TFile.Open(mcfile)
cbmsim=mcfile.cbmsim

In [ ]:
ROOT.gStyle.SetOptStat(1111)

hnuet = ROOT.TH1D("nuet", "MC slope;tan#theta", 1500, 0, 1.5)
hpi0t = ROOT.TH1D("pi0t", "MC pi0 slope;tan#theta", 1500, 0,1.5)

cut = "TMath::Abs(MCTrack[1].fPdgCode)==11&&TMath::Abs(MCTrack[0].fPdgCode)==12"
cutpi = "MCTrack.fPdgCode==111&&MCTrack.fMotherId==0"
     
cbmsim.Draw("TMath::Sqrt((MCTrack[1].fPx/MCTrack[1].fPz)^2+(MCTrack[1].fPy/MCTrack[1].fPz)^2)>>nuet", cut)
cbmsim.Draw("TMath::Sqrt((MCTrack.fPx/MCTrack.fPz)^2+(MCTrack.fPy/MCTrack.fPz)^2)>>pi0t", cutpi)

In [ ]:
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)
ROOT.gStyle.SetFrameLineWidth(2)
ctheta = ROOT.TCanvas("ctheta","ctheta",1500,600)
ctheta.Divide(2,1)

leg = ROOT.TLegend(0.65, 0.7, 0.85, 0.85)
leg.AddEntry(hnuet, 'e', 'L')
leg.AddEntry(hpi0t, '#pi^{0}', 'L')
leg.SetTextFont(53)
leg.SetBorderSize(0)

l = ROOT.TLine(0.0036, 0.0001, 0.0036, 0.043)
l.SetLineWidth(3)
l.SetLineColor(8)


ctheta.cd(1)
hnuet.GetXaxis().SetRangeUser(0, 1.5)
hnuet.GetXaxis().SetLabelFont(43)
hnuet.GetXaxis().SetLabelSize(22)
hnuet.GetXaxis().SetTitleOffset(0.7)
hnuet.GetXaxis().SetTitleSize(24)
hnuet.GetXaxis().SetTitleFont(63)
hnuet.SetLineWidth(1)
hnuet.DrawNormalized()
hpi0t.SetLineWidth(1)
hpi0t.SetLineColor(2)
hpi0t.DrawNormalized("sames&&hist")
leg.Draw("same")

ctheta.cd(2)
hnuet.GetXaxis().SetRangeUser(0, 0.1)
hnuet.GetXaxis().SetNdivisions(510)
hnuet.SetLineWidth(2)
#hnuet.Rebin(2)
hnuet.DrawNormalized()
hpi0t.SetLineWidth(2)
hpi0t.SetLineColor(2)
#hpi0t.Rebin(2)
hpi0t.DrawNormalized("sames&&hist")
l.Draw("same")
leg.Draw("same")

from ctypes import c_int
plate=24
cx = ROOT.TCanvas("cx","cx",500,500)
_sliceMax=c_int(0)
_sliceMay=c_int(0)
_sliceMaz=c_int(0)
hist_cp[f'h_{plate}'].GetXaxis().SetRangeUser(11500,14500)
hist_cp[f'h_{plate}'].GetYaxis().SetRangeUser(104000,107500)
hist_cp[f'h_{plate}'].GetMaximumBin(_sliceMax, _sliceMay, _sliceMaz)
sliceMax=_sliceMax.value
ctheta.Draw()

In [ ]:
sx = 15 - 17
sy = 35 - 32
sxy = ROOT.TMath.Sqrt(sx**2+sy**2)
print(sxy)

In [ ]:
binIndex = 8
binLowEdge = hnuet.GetXaxis().GetBinLowEdge(binIndex)
binUpEdge = hnuet.GetXaxis().GetBinUpEdge(binIndex)
print(binLowEdge)

In [ ]:
from ctypes import c_double

_errnue=c_double(0)
_errpi=c_double(0)

probnue = hnuet.IntegralAndError(8,hnuet.FindLastBinAbove(), _errnue)/hnuet.GetEntries()
probpi0 = hpi0t.IntegralAndError(8,hpi0t.FindLastBinAbove(), _errpi)/hpi0t.GetEntries()

errnue=_errnue.value/hnuet.GetEntries()
errpi=_errpi.value/hnuet.GetEntries()
print(errnue, errpi)
print(round(probnue,2), round(probpi0,2))

In [ ]:
probnue = hnuet.Integral(1,601)/hnuet.GetEntries()
probpi0 = hpi0t.Integral(1,600)/hpi0t.GetEntries()

print(round(probnue,3), round(probpi0,3))

In [ ]:
dz = [289.3743, 353.3114, 31.9685]
dx = [-47.7758,  -5.5467, 21.1145]
dy = [13.0335 ,  55.2793, 21.1229]
dz21 = [303.8430, 311.6324, 3.8947]
dx21 = [-26.6035,  -7.3039, 9.6498]
dy21 = [15.0747 ,  34.3728, 9.6490]
cutpdg = "TMath::Abs(MCTrack[0].fPdgCode)==12"
cutpos = f"TMath::Abs(MCTrack[1].fStartX-{dx[0]+dx[2]})<{dx[2]}&&TMath::Abs(MCTrack[1].fStartY-{dy[0]+dy[2]})<{dy[2]}&&TMath::Abs(MCTrack[1].fStartZ-{dz[0]+dz[2]})<{dz[2]}"
cutpos21 = f"TMath::Abs(MCTrack[1].fStartX-{dx21[0]+dx21[2]})<{dx21[2]}&&TMath::Abs(MCTrack[1].fStartY-{dy21[0]+dy21[2]})<{dy21[2]}&&TMath::Abs(MCTrack[1].fStartZ-{dz21[0]+dz21[2]})<{dz21[2]}"
cut = cutpdg+"&&"+cutpos
cut21 = cutpdg+"&&"+cutpos21
fact=cbmsim.GetEntries(cut21)/cbmsim.GetEntries(cut)
print(fact, 1/20)
print(cbmsim.GetEntries(cut21))

In [ ]:
nutot= (46+14.6)/8.6*9.5
nuecc= 10/8.6*9.5
numucc=36/8.6*9.5
nutot21=nutot*fact
nuecc21=nuecc*fact
numucc21=numucc*fact
print(nutot, nutot21, nuecc,nuecc21,numucc,numucc21)

In [ ]:
z=  321.3429
dZ= [289.3743, 353.3114]
dx= [-47.7758,-5.5467]
dy= [13.0335, 55.2793] 
#Brick_1
z=  307.7377
dZ= [303.8430, 311.6324]
dx= [-26.6035, -7.3039]
dy= [15.0747, 34.3728]